# Stop Trader

The goal of the `stop` trader is to explore whether it makes sense to set up stop-losses (or stop-buys) when entering a long position (or when selling everything).

Imagine that you buy a certain amount $a$ of a cryptocurrency (e.g. `BTC`) at price $p$. Is it a good idea to set up a stop-loss at some lower price $p' < p$? If so, how to determine the price $p'$? What if later the price $p$ goes up? Should we then adjust (increase) the stop-loss price $p'$ as well? These are some of the questions that we would like to answer by evaluating many instances of the `stop` trader over the `BTCUSD` price history.

At any given moment the trader has (on its account) some amount $b$ of a base (crypto)currency (e.g. `BTC`) and some amount $q$ of a quote currency (e.g. `USD`). Let $p$ denote the base (crypto)currency price (e.g. `BTCUSD` price). If $b * p \ge q$ we say that the trader is in the `LONG` state. Otherwise, we say that the trader is in the `CASH` state.

* If the trader gets into the `LONG` state we define the stop-loss at the price $p' := (1 - \alpha) * p < p$, where $\alpha \in (0; 1)$ is a trader hyper-parameter (called `stop_order_margin`). As long as the trader is in the `LONG` state we only allow the price of the stop-loss $p'$ to increase over time. When the price $p$ drops and hits the stop-loss the trader automatically switches to the `CASH` state (by selling all its $b$ base (crypto)currency at price $p'$).
* Similarly, if the trader gets into the `CASH` state we define the stop-buy at the price $p' := (1 + \alpha) * p > p$. As long as the trader is in the `CASH` state we only allow the price of the stop-buy $p'$ to decrease over time. When the price $p$ increases and hits the stop-buy the trader automatically switches to the `LONG` state (by buying as much as possible base (crypto)currency at price $p'$).

One can easily observe that if there were no transaction fees and if we could always execute the stop-sells (stop-buys) at exactly the specified price $p'$, this strategy would never be worse than the Buy-And-HODL strategy. Moreover, we would (under these ideal circumstances) never loose any money (only potentially gain). Note: If you think that these assumptions (no transaction fees etc.) are far-fetched then take a look at the assumptions behind the famous [Black-Scholes model](https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model) (they are even more unrealistic).

Unfortunately, we do have non-negligible transaction fees and the stop orders are (usually) not executed at exactly the desired price $p'$, but at a worse price (especially if there is not enough liquidity). Therefore, we need to tune the hyper-parameters of the trader so that we do not incur too many transaction fees and still stay profitable.

In addition to the above-mentioned `stop_order_margin` $\alpha$ we have the following hyper-parameters:

* `stop_order_move_margin` $\beta$: (A) If the trader is in the `LONG` state and the stop-loss price: $p' \le (1 - \beta) * p$ we allow the stop-loss price to increase. (B) Similarly, if the trader is in the `CASH` state and the stop-buy price $p' \ge (1 + \beta) * p$ we allow the stop-buy price to decrease.
* `stop_order_increase_per_day` $\gamma_+$: In the case (A) we allow the stop-loss price to increase by at most $\gamma_+ * 100\%$ per day, although we do not allow the stop-loss price to exceed the $(1 - \beta) * p$ threshold.
* `stop_order_decrease_per_day` $\gamma_-$: In the case (B) we allow the stop-buy price to decrease by at most $\gamma_- * 100\%$ per day, although we do not allow the stop-buy price to drop below the $(1 + \beta) * p$ threshold.

Batch evaluation of the `stop` trader reveals that setting $\alpha = \beta = 0.1$, $\gamma_+ = 0.01$, and $\gamma_- = 0.1$ gives a reasonable performance (although there are many other settings which perform similarly). With these specific parameters the trader performs as follows (evaluated over 6 month time periods within the 4 year time period: `[2017-01-01 - 2021-01-01)` with 5 minute sampling rate):

```
bazel run :trader -- \
  --input_ohlc_history_delimited_proto_file="/$(pwd)/data/bitstampUSD_5min.dpb" \
  --trader="stop" \
  --start_date_utc="2017-01-01" \
  --end_date_utc="2021-01-01" \
  --evaluation_period_months=6 \
  --start_base_balance=1.0 \
  --start_quote_balance=0.0
```

Output (shortened):

```
...
Reading OHLC history from: .../data/bitstampUSD_5min.dpb
- Loaded 420768 records in 0.782 seconds
- Selected 420768 records within the time period: [2017-01-01 00:00:00 - 2021-01-01 00:00:00)

stop-trader[0.100|0.100|0.010|0.100] evaluation:
------------------ period ------------------    trader & base gain    score    t&b volatility
[2017-01-01 00:00:00 - 2017-07-01 00:00:00):     90.13%    155.13%    0.745    0.744    0.767
[2017-02-01 00:00:00 - 2017-08-01 00:00:00):    159.53%    196.26%    0.876    0.768    0.845
[2017-03-01 00:00:00 - 2017-09-01 00:00:00):    255.64%    297.85%    0.894    0.801    0.877
[2017-04-01 00:00:00 - 2017-10-01 00:00:00):    314.13%    302.96%    1.028    0.846    0.921
[2017-05-01 00:00:00 - 2017-11-01 00:00:00):    393.33%    376.78%    1.035    0.872    0.946
...
[2020-03-01 00:00:00 - 2020-09-01 00:00:00):     59.34%     36.01%    1.171    0.651    0.982
[2020-04-01 00:00:00 - 2020-10-01 00:00:00):     44.16%     67.60%    0.860    0.546    0.571
[2020-05-01 00:00:00 - 2020-11-01 00:00:00):      8.95%     59.90%    0.681    0.501    0.512
[2020-06-01 00:00:00 - 2020-12-01 00:00:00):     79.72%    108.97%    0.860    0.490    0.517
[2020-07-01 00:00:00 - 2021-01-01 00:00:00):    206.57%    218.20%    0.963    0.522    0.542

Evaluated in 10.021 seconds

```

As you can see, the `stop` trader sometimes beats the Buy-And-HODL strategy (although not always). Over the whole 4 year time period: `[2017-01-01 - 2021-01-01)` with 5 minute sampling rate:

```
...
Reading OHLC history from: .../data/bitstampUSD_5min.dpb
- Loaded 420768 records in 1.064 seconds
- Selected 420768 records within the time period: [2017-01-01 00:00:00 - 2021-01-01 00:00:00)

stop-trader[0.100|0.100|0.010|0.100] evaluation:
------------------ period ------------------    trader & base gain    score    t&b volatility
[2017-01-01 00:00:00 - 2021-01-01 00:00:00):   3220.23%   2900.17%    1.107    0.713    0.828

Evaluated in 1.842 seconds
```

the `stop` trader would beat the Buy-And-HODL strategy only by 10%. That being said, there are many other (suboptimal) choices of the hyper-parameters that would lead to much worse performance. Thus it is questionable whether an average person would be able to set the stop-losses / stop-buys better than this algorithm. If not then an average person would be better off with HODLing without any stop-losses.